## Configuraciones comunes para un ambiente de `MLflow Tracking`.
![](https://mlflow.org/docs/latest/_images/tracking-setup-overview.png)

### Escenario 2
```python
# set mlflow tracking uri
import mlflow
mlflow.set_tracking_uri('sqlite:///mlflow.db')
```

### Escenario 3
MLFlow remoto

```python
import mlflow

mlflow.set_tracking_uri('url/remote/server')
```


## `MLflow`: Beneficios
* El `Tracking server` puede ser fácilmente desplegado en la nube
* Compartir experimentos con otros Data Scientists
* Colaborar con otros para construir y desplegar modelos
* Dar más visibilidad de los esfuerzos del equipo de Data Science.

## `MLflow`: Problemas cuando se ejecutan servidores remotos compartidos
* Seguridad:
    * Restringir el acceso al server (por ejemplo a través de una VPN)
* Isolation:
    * Definir un estándar para nombrar experimentos, modelos y un conjunto de tags predeterminados.
    * Restringir el acceso a los artefactos  

## `MLflow`: Limitaciones
* **Autenticación y Usuarios:** La versión open source de `MLflow` no provee ningún tipo de autenticación
* **Versionamiento de datos** 
    * Para asegurar total reproducibilidad, necesitamos versionar los datos que se usan para entrenar el modelo.
    * `MLflow` no provee una solución para eso, pero hay maneras de mitigarlo
* **Monitoreo del modelo y datos:** Veremos la herramienta adecuada para este fin 

# DagsHub
 <div style="text-align:center">
    <img src="https://user-images.githubusercontent.com/611655/181510038-e38f4001-c304-411e-8f45-f71554eb9763.png" alt="DagsHub Logo">
</div>

## Introducción:
DagsHub es una plataforma revolucionaria que se describe como el "GitHub para el aprendizaje automático". Permite a los científicos de datos y desarrolladores de aprendizaje automático gestionar y colaborar en sus proyectos de manera eficiente, asegurando la reproducibilidad y el control de versiones.

## Características Clave:
1. **Control de Versiones**: Realiza un seguimiento de los cambios en los datos, el código y los modelos, garantizando un historial completo de tu proyecto de aprendizaje automático.
2. **Colaboración**: Facilita la colaboración dentro de los equipos al permitir que varios usuarios trabajen en el mismo proyecto manteniendo el historial de versiones.
3. **Versionado de Datos**: Realiza un seguimiento de las versiones de los datos, lo que facilita la reproducción de experimentos y el intercambio de conjuntos de datos.
4. **Reproducibilidad**: Asegura que los experimentos se puedan replicar con el mismo código, datos y entorno.
5. **Interfaz Web**: Ofrece una interfaz web intuitiva para organizar y gestionar proyectos de aprendizaje automático.
6. **Repositorios Públicos y Privados**: Ofrece tanto repositorios públicos como privados para compartir proyectos de manera abierta o segura.
7. **Seguimiento de Experimentos**: Registra todos los detalles de los experimentos de aprendizaje automático, lo que facilita el análisis y la comparación de resultados.
8. **Integración**: Se integra fácilmente con herramientas y formatos de código abierto populares, como Jupyter notebooks y Git.
9. **Organización de Proyectos**: Proporciona herramientas para mantener estructurado y bien documentado tu proyecto de aprendizaje automático.


## Dagshub

1. Creamos una cuenta [aquí](https://dagshub.com/user/sign_up). Se puede asociar con la cuenta de GitHub.
2. Cambiar contraseña.
3. Crear un primer repositorio.

## Actividad
Vamos a prepara el ambiente de trabajo para la siguiente clase:

1. Creamos un repositorio en `Github` llamado `nyc-taxi-time-prediction`
2. Vinculamos el repositorio a nuestra cuenta de `Dagshub`
3. Clonamos el repositorio de `Github` en nuestro local
4. Creamos un ambiente virtual
5. Crear una branch `experiments`
6. Crear un directorio `experiments` en la carpeta raíz del proyecto
7. Crer un `jupyter-notebook` dentro de dicho directorio con el nombre `model_experiments.ipynb`

![ml flow cheatsheet](images/mlflow-cheatsheet.png)


## Vamos a reutilizar el código que ya hemos usado

```bash
pip install mlflow==2.16.1 dagshub==0.3.35 jupyter==1.1.1 xgboost==2.1.1 hyperopt==0.2.7
```

Copiar dataset en una carpeta `data`

In [ ]:
# Create the directory if it doesn't exist
!mkdir -p ../data

# Download files using curl
!curl -o ../data/green_tripdata_2024-01.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-01.parquet
!curl -o ../data/green_tripdata_2024-02.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-02.parquet

Importar las librerías necesarias y definir función para importar los datos

In [ ]:
import pickle
import pandas as pd
from sklearn.metrics import  root_mean_squared_error
from sklearn.feature_extraction import  DictVectorizer
from sklearn.linear_model import Lasso, Ridge, LinearRegression

In [ ]:
def read_dataframe(filename):

    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

In [ ]:
df_train = read_dataframe('../data/green_tripdata_2024-01.parquet')
df_val = read_dataframe('../data/green_tripdata_2024-02.parquet')

Feature Engineering

In [ ]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

One Hot Encoding

In [ ]:
categorical = ['PU_DO']  #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']
dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [ ]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

Definir el `tracking URI` y el nombre del experimento

In [ ]:
import dagshub
import mlflow


dagshub.init(url="<url/del/repo>", mlflow=True)

MLFLOW_TRACKING_URI = mlflow.get_tracking_uri()

print(MLFLOW_TRACKING_URI)

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(experiment_name="nyc-taxi-experiment")

Definir los `dataset` como objetos de `mlflow` para poderlos trackear

In [ ]:
training_dataset = mlflow.data.from_numpy(X_train.data, targets=y_train, name="green_tripdata_2024-01")
validation_dataset = mlflow.data.from_numpy(X_val.data, targets=y_val, name="green_tripdata_2024-02")

### Subir los dataset al storage que nos brinda `dagshub`

Ahora vamos a entrenar un modelo `xgboost`


In [ ]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
import pathlib

Definir los `dataset` a trabajar.

In [ ]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

Definir la función objetivo

In [ ]:
def objective(params):
    with mlflow.start_run(nested=True):
         
        # Tag model
        mlflow.set_tag("model_family", "xgboost")
        
        # Log parameters
        mlflow.log_params(params)
        
        # Train model
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=100,
            evals=[(valid, 'validation')],
            early_stopping_rounds=10
        )
        
        # Log xgboost model with artifact_path
        mlflow.xgboost.log_model(booster, artifact_path="model")
         
        # Predict in the val dataset
        y_pred = booster.predict(valid)
        
        # Calculate metric
        rmse = root_mean_squared_error(y_val, y_pred)
        
        # Log performance metric
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

Definir el espacio de búsqueda

In [ ]:
mlflow.xgboost.autolog()

with mlflow.start_run(run_name="Xgboost Hyper-parameter Optimization", nested=True):
    search_space = {
        'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
        'learning_rate': hp.loguniform('learning_rate', -3, 0),
        'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
        'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
        'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
        'objective': 'reg:squarederror',
        'seed': 42
    }
    
    best_params = fmin(
        fn=objective,
        space=search_space,
        algo=tpe.suggest,
        max_evals=10,
        trials=Trials()
    )
    best_params["max_depth"] = int(best_params["max_depth"])
    best_params["seed"] = 42
    best_params["objective"] = "reg:squarederror"
    
    mlflow.log_params(best_params)

    # Log tags
    mlflow.set_tags(
        tags={
            "project": "NYC Taxi Time Prediction Project",
            "optimizer_engine": "hyper-opt",
            "model_family": "xgboost",
            "feature_set_version": 1,
        }
    )

    # Log a fit model instance
    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=100,
        evals=[(valid, 'validation')],
        early_stopping_rounds=10
    )
        
    y_pred = booster.predict(valid)
    
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)
    
    pathlib.Path("models").mkdir(exist_ok=True)
    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
        
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

In [ ]:
best_params

Ahora vamos a registrar el mejor modelo en el `model registry` y usarlo para hacer predicciones

In [ ]:
run_id = input("Ingrese el run_id")
run_uri = f"runs:/{run_id}/model"

result = mlflow.register_model(
    model_uri=run_uri,
    name="nyc-taxi-model"
)

In [ ]:
from datetime import datetime
from mlflow import MlflowClient

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)
client.update_registered_model(
    name="nyc-taxi-model",
    description="Model registry for the NYC Taxi Time Prediction Project",
)

new_alias = "champion"
date = datetime.today()
model_version = "1"

# create "champion" alias for version 1 of model "nyc-taxi-model"
client.set_registered_model_alias(
    name="nyc-taxi-model",
    alias=new_alias,
    version=model_version
)

client.update_model_version(
    name="nyc-taxi-model",
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_alias} on {date}",
)

In [ ]:
import mlflow.pyfunc

model_name = "nyc-taxi-model"
alias = "champion"

model_uri = f"models:/{model_name}@{alias}"

champion_version = mlflow.pyfunc.load_model(
    model_uri=model_uri
)

champion_version.predict(X_val)

## Tarea y actividad en clase.

1. Hacer merge de la rama que trabajamos a main.
2. Crear una nueva rama que se llame `feat: tarea 5`.
3. Crear un nuevo `jupyter-notebook` llamado `challenger-experiments.ipynb` en la rama creada anteriormente
4. Hacer dos `parent experiments` con `Gradient Boost` y `Random Forest` regressors en donde cada uno tenga `child experiments` con búsqueda de hyper-parámetros. Puede usar cualquier libreraría con la que se sienta cómodo: `hyperopt`, `optuna`, `scikit-learn` (Grid Search, Random Search, Halving Search etc)
5. Registrar el modelo con la mejor métrica de los obtenidos en dichos experimentos en el `model registry` en el mismo modelo ya previamente creado `nyc-taxi-model`.
6. Asígnele el alias `challenger`
7. Descargue en la carpeta `data` el conjunto de datos correspondiente a marzo del 2024
8. Guardela en el `storage` disponible de `mlflow`
9. Use ese conjunto de datos para probarlo sobre los modelos con el alias `champion` y `challenger`
10. Obtenga la métrica de cada modelo
11. Decida si el nuevo modelo `challenger` debe ser promovido a `champion` o no. Use los criterios que usted como Data Scientis considere relevantes y justifique la respuesta.
12. Abrir un `PR` con los cambios hechos en la rama `feat: tarea 5` hacia la rama `main`.


Habrá dos entregas divididas de la siguiente manera:

1. **Trabajo en clase hoy Martes 17 de Septiembre de 2024.** Para esta entrega, hacer un commit con el siguiente mensaje `feat: entrega trabajo en clase` con los avances realizados en clase.

2. **Tarea: Viernes 20 de Septiembre de 2024 a las 19:55.** Esta entrega debe contener todo lo descrito anteriormente